### Importing the neaded libraries
### Setting up dataset and JSON file

In [2]:
import json
import os
import math
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

dataset = "coins"
json_file = "data.json"

###  Preparing the dataset

In [3]:
sr= 22050
duration = 30
samples = sr * duration

def extract_mfcc(dataset, file_json, num_mfcc=40, n_fft=1024, hop_length=512, num_segments=5):
    
    data = { "coin": [], "labels": [], "mfcc": []}
    samples_per_segment = int(samples / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)
    
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset)):
        if dirpath is not dataset:
            semantic_label = dirpath.split("/")[-1]
            data["coin"].append(semantic_label)
            print("\nProcessing: {}".format(semantic_label))
            
            for f in filenames:
                file_path = os.path.join(dirpath, f)
                signal, sample_rate = librosa.load(file_path, sr=sr)
                for d in range(num_segments):
                    start = samples_per_segment * d
                    finish = start + samples_per_segment
                    
                    mfcc = librosa.feature.mfcc(y=signal[start:finish], sr=sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                    mfcc = mfcc.T

                    if len(mfcc) == num_mfcc_vectors_per_segment:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(i-1)
                        print("{}, segment:{}".format(file_path, d+1))
                        

    with open(file_json, "w") as fp:
        json.dump(data, fp, indent=4)

In [4]:
extract_mfcc(dataset, json_file,  num_segments=20)


Processing: coins\0.020 DT
coins\0.020 DT\New Recording 43.wav, segment:1
coins\0.020 DT\New Recording 43.wav, segment:2
coins\0.020 DT\New Recording 46(1).wav, segment:1
coins\0.020 DT\New Recording 46(1).wav, segment:2
coins\0.020 DT\New Recording 46.wav, segment:1
coins\0.020 DT\New Recording 46.wav, segment:2


C:\Users\myrai\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=0
  warnings.warn(


coins\0.020 DT\New Recording 47(1).wav, segment:1
coins\0.020 DT\New Recording 47(1).wav, segment:2
coins\0.020 DT\New Recording 47.wav, segment:1
coins\0.020 DT\New Recording 47.wav, segment:2
coins\0.020 DT\New Recording 48(1).wav, segment:1
coins\0.020 DT\New Recording 48(1).wav, segment:2
coins\0.020 DT\New Recording 48.wav, segment:1
coins\0.020 DT\New Recording 48.wav, segment:2
coins\0.020 DT\New Recording 49(1).wav, segment:1
coins\0.020 DT\New Recording 49(1).wav, segment:2
coins\0.020 DT\New Recording 49.wav, segment:1
coins\0.020 DT\New Recording 49.wav, segment:2
coins\0.020 DT\New Recording 50(1).wav, segment:1
coins\0.020 DT\New Recording 50(1).wav, segment:2
coins\0.020 DT\New Recording 50.wav, segment:1
coins\0.020 DT\New Recording 50.wav, segment:2
coins\0.020 DT\New Recording 51(1).wav, segment:1
coins\0.020 DT\New Recording 51(1).wav, segment:2
coins\0.020 DT\New Recording 51.wav, segment:1
coins\0.020 DT\New Recording 51.wav, segment:2
coins\0.020 DT\New Recording 5

C:\Users\myrai\anaconda3\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=169
  warnings.warn(


coins\0.100 DT\New Recording 27(1).wav, segment:1
coins\0.100 DT\New Recording 27(1).wav, segment:2
coins\0.100 DT\New Recording 27.wav, segment:1
coins\0.100 DT\New Recording 27.wav, segment:2
coins\0.100 DT\New Recording 28.wav, segment:1
coins\0.100 DT\New Recording 28.wav, segment:2
coins\0.100 DT\New Recording 29(1).wav, segment:1
coins\0.100 DT\New Recording 29(1).wav, segment:2
coins\0.100 DT\New Recording 29.wav, segment:1
coins\0.100 DT\New Recording 29.wav, segment:2
coins\0.100 DT\New Recording 31(1).wav, segment:1
coins\0.100 DT\New Recording 31(1).wav, segment:2
coins\0.100 DT\New Recording 31.wav, segment:1
coins\0.100 DT\New Recording 31.wav, segment:2
coins\0.100 DT\New Recording 32(1).wav, segment:1
coins\0.100 DT\New Recording 32(1).wav, segment:2
coins\0.100 DT\New Recording 32.wav, segment:1
coins\0.100 DT\New Recording 32.wav, segment:2
coins\0.100 DT\New Recording 33(1).wav, segment:1
coins\0.100 DT\New Recording 33(1).wav, segment:2
coins\0.100 DT\New Recording 3

### function for loading files from the data 

In [13]:
def load_data(dataset):
    with open(dataset, "r") as file:
        data = json.load(file)
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    print("Data loaded!")

    return X, y

### Loading and splitting the dataset
### Defining the neural network model
### Displaying model summary
### Training the model + solving overfetting 

In [7]:
X, y = load_data(json_file)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = keras.Sequential([

        keras.layers.Flatten(input_shape=(X.shape[1], X.shape[2])),
    
        keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),
    
        keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),
    
        keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
        keras.layers.Dropout(0.3),

        keras.layers.Dense(3, activation='softmax')
    ])

optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
 
model.summary()

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=100)

Data loaded!
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 2600)              0         
                                                                 
 dense_4 (Dense)             (None, 512)               1331712   
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 64)                16448     
                                                                 
 dropout_5 (Dropout)         (None, 64)  

Epoch 48/100
3/3 [==============================] - 0s 62ms/step - loss: 7.9863 - accuracy: 0.5632 - val_loss: 2.0929 - val_accuracy: 0.6364
Epoch 49/100
3/3 [==============================] - 0s 91ms/step - loss: 5.5426 - accuracy: 0.6782 - val_loss: 2.0730 - val_accuracy: 0.7273
Epoch 50/100
3/3 [==============================] - 0s 98ms/step - loss: 8.3397 - accuracy: 0.5632 - val_loss: 2.1518 - val_accuracy: 0.7273
Epoch 51/100
3/3 [==============================] - 0s 59ms/step - loss: 9.2580 - accuracy: 0.5747 - val_loss: 2.2249 - val_accuracy: 0.7273
Epoch 52/100
3/3 [==============================] - 0s 73ms/step - loss: 4.2430 - accuracy: 0.7701 - val_loss: 2.2970 - val_accuracy: 0.6818
Epoch 53/100
3/3 [==============================] - 0s 67ms/step - loss: 8.5061 - accuracy: 0.6437 - val_loss: 2.4975 - val_accuracy: 0.6364
Epoch 54/100
3/3 [==============================] - 0s 69ms/step - loss: 6.7017 - accuracy: 0.6552 - val_loss: 2.4844 - val_accuracy: 0.6364
Epoch 55/100


In [31]:
def preprocess_audio(file_path):
    signal, sr = librosa.load(file_path, sr=22050)
    mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=40, hop_length=512, n_fft=1024)
    expected_time_steps = 65
    if mfcc.shape[1] < expected_time_steps:
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, expected_time_steps - mfcc.shape[1])), mode='constant')
        print(mfcc.shape[1])
    else:
        mfcc = mfcc[:, :expected_time_steps]
        print(mfcc.shape[1])
    return mfcc.T

In [39]:
with open(json_file, "r") as file:
        data = json.load(file)

In [40]:
audio_file_path = "0.5dt.wav"
input_features = preprocess_audio(audio_file_path)
input_features = np.expand_dims(input_features, axis=0)

predictions = model.predict(input_features)


predicted_class = np.argmax(predictions)


print(f"Predicted Class: {predicted_class}")
print(f"Class Name: {data['coin'][predicted_class]}")

65
1/1 [==============================] - 0s 55ms/step
Predicted Class: 2
Class Name: coins\0.500 DT


In [41]:
audio_file_path = "0.1dt.wav"
input_features = preprocess_audio(audio_file_path)
input_features = np.expand_dims(input_features, axis=0)

predictions = model.predict(input_features)


predicted_class = np.argmax(predictions)

print(f"Predicted Class: {predicted_class}")
print(f"Class Name: {data['coin'][predicted_class]}")

65
1/1 [==============================] - 0s 40ms/step
Predicted Class: 1
Class Name: coins\0.100 DT


In [42]:
audio_file_path = "0.02dt.wav"
input_features = preprocess_audio(audio_file_path)
input_features = np.expand_dims(input_features, axis=0)

predictions = model.predict(input_features)


predicted_class = np.argmax(predictions)

print(f"Predicted Class: {predicted_class}")
print(f"Class Name: {data['coin'][predicted_class]}")

65
1/1 [==============================] - 0s 52ms/step
Predicted Class: 0
Class Name: coins\0.020 DT
